In [ ]:
tokenizer = AutoTokenizer.from_pretrained("asafaya/bert-base-arabic")
model = AutoModel.from_pretrained("asafaya/bert-base-arabic")

def get_embeddings(text):
    tokens = tokenizer(text, return_tensors="pt", truncation=True, max_length=512)
    model_output = model(**tokens)

    return model_output.last_hidden_state.detach().numpy()

# Assuming list_of_string_sentences is a list of undiacritized sentences
undiacritized_sentences = list_of_string_sentences

# Create a list of tuples, where each tuple contains an undiacritized sentence and its diacritized version
paired_sentences = [(undiacritized_sentence, diacritized_sentence) for undiacritized_sentence, diacritized_sentence in zip(undiacritized_sentences, diacritized_sentences)]

# Get embeddings for each paired sentence
paired_sentences_embeddings = [(get_embeddings(undiacritized_sentence), diacritized_sentence) for undiacritized_sentence, diacritized_sentence in paired_sentences]

print(len(paired_sentences_embeddings))


In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForTokenClassification, AdamW

# Sample data (replace with your actual data)
train_data = [("سلام", "سَلام"), ("مرحبا", "مَرحبًا"), ...]
dev_data = [("مرحبا", "مَرحبًا"), ...]
test_data = [("أهلا", ""), ...]

# Load pre-trained model and tokenizer
model_name = "asafaya/bert-base-arabic"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForTokenClassification.from_pretrained(model_name, num_labels=len(tokenizer))

# Tokenize and encode data
train_encodings = tokenizer(train_data, return_tensors="pt", padding="max_length", truncation=True)
dev_encodings = tokenizer(dev_data, return_tensors="pt", padding="max_length", truncation=True)
test_encodings = tokenizer(test_data, return_tensors="pt", padding="max_length", truncation=True)

# Extract labels from encoded data
train_labels = torch.tensor([[label_ids] for label_ids in tokenizer(train_data[1], return_tensors="pt")["input_ids"]])
dev_labels = torch.tensor([[label_ids] for label_ids in tokenizer(dev_data[1], return_tensors="pt")["input_ids"]])

# Fine-tune the model
optimizer = AdamW(model.parameters(), lr=2e-5)
for epoch in range(3):
    optimizer.zero_grad()
    outputs = model(**train_encodings, labels=train_labels)
    loss = outputs.loss
    loss.backward()
    optimizer.step()

# Evaluate on dev set
with torch.no_grad():
    outputs = model(**dev_encodings, labels=dev_labels)
    # Process outputs for evaluation

# Test the model on the test set
with torch.no_grad():
    test_outputs = model(**test_encodings)
    # Process test_outputs for testing
